In [0]:
# Notebook parameters (you can edit these later)
RUN_LABEL = "manual"      # e.g., "daily", "manual", "backfill"
SEARCH_TERMS = [
    "data analyst",
    "business analyst",
    "data scientist",
    "analytics engineer",
    "bi analyst",
]

BRONZE_DB  = "job_market"                  # database (schema)
BRONZE_TBL = "jobs_bronze"                 # table name
RAW_DIR    = "/dbfs/FileStore/job_market/raw"   # where we also save raw JSON snapshots


In [0]:
import os, time, json, requests
from datetime import datetime
from urllib.parse import urlencode

from pyspark.sql import functions as F, types as T, Window


In [0]:
import os
from datetime import datetime

def log(msg):
    print(f"[{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC] {msg}")

BRONZE_DB = "bronze_db"
RAW_DIR = "/tmp/job_data/raw"  # Writable in Community Edition

# Create the database
spark.sql(f"CREATE DATABASE IF NOT EXISTS {BRONZE_DB}")
os.makedirs(RAW_DIR, exist_ok=True)

log("Bronze layer initialized.")



[2025-08-27 23:00:34 UTC] Bronze layer initialized.


/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071165-604942667:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC] {msg}")


In [0]:
BASE_URL = "https://www.themuse.com/api/public/jobs"

def fetch_page(page:int, query:str):
    params = {"page": page, "descending": "true", "category": "Data and Analytics", "query": query}
    url = f"{BASE_URL}?{urlencode(params)}"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()

def fetch_all(terms, sleep_sec=0.3, max_pages=50):
    items = []
    for term in terms:
        log(f"Fetching: {term}")
        page = 1
        while page <= max_pages:
            data = fetch_page(page, term)
            results = data.get("results", [])
            if not results:
                break
            items.extend(results)
            log(f" - page {page}, items {len(results)}")
            page += 1
            time.sleep(sleep_sec)
    # de-dup by API id across all terms
    dedup, seen = [], set()
    for it in items:
        jid = it.get("id")
        if jid in seen: 
            continue
        seen.add(jid)
        dedup.append(it)
    return dedup

raw_items = fetch_all(SEARCH_TERMS)
log(f"Total unique postings fetched: {len(raw_items)}")


/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071165-604942667:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC] {msg}")


[2025-08-27 23:00:41 UTC] Fetching: data analyst
[2025-08-27 23:00:41 UTC]  - page 1, items 20
[2025-08-27 23:00:41 UTC]  - page 2, items 20
[2025-08-27 23:00:42 UTC]  - page 3, items 20
[2025-08-27 23:00:42 UTC]  - page 4, items 20
[2025-08-27 23:00:43 UTC]  - page 5, items 20
[2025-08-27 23:00:43 UTC]  - page 6, items 20
[2025-08-27 23:00:44 UTC]  - page 7, items 20
[2025-08-27 23:00:44 UTC]  - page 8, items 20
[2025-08-27 23:00:44 UTC]  - page 9, items 20
[2025-08-27 23:00:45 UTC]  - page 10, items 20
[2025-08-27 23:00:45 UTC]  - page 11, items 20
[2025-08-27 23:00:46 UTC]  - page 12, items 20
[2025-08-27 23:00:46 UTC]  - page 13, items 20
[2025-08-27 23:00:47 UTC]  - page 14, items 20
[2025-08-27 23:00:47 UTC]  - page 15, items 20
[2025-08-27 23:00:48 UTC]  - page 16, items 20
[2025-08-27 23:00:48 UTC]  - page 17, items 20
[2025-08-27 23:00:49 UTC]  - page 18, items 20
[2025-08-27 23:00:49 UTC]  - page 19, items 20
[2025-08-27 23:00:50 UTC]  - page 20, items 20
[2025-08-27 23:00:50

In [0]:
ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
raw_path = f"{RAW_DIR}/themuse_{ts}_{RUN_LABEL}.json"
with open(raw_path, "w", encoding="utf-8") as f:
    json.dump(raw_items, f, ensure_ascii=False, indent=2)

log(f"Saved raw JSON snapshot to: {raw_path}")


[2025-08-27 23:02:50 UTC] Saved raw JSON snapshot to: /tmp/job_data/raw/themuse_20250827_230250_manual.json


/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071167-4035765018:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071165-604942667:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC] {msg}")


In [0]:
def safe_join(l, key="name"):
    if not l: 
        return None
    return "; ".join([x.get(key) for x in l if x and x.get(key)])

rows = []
for it in raw_items:
    rows.append({
        "job_id": it.get("id"),
        "title_raw": (it.get("name") or "").strip(),
        "company":   ((it.get("company") or {}).get("name")),
        "location_raw": safe_join(it.get("locations") or []),
        "post_date": (it.get("publication_date") or "")[:10],
        "url":       ((it.get("refs") or {}).get("landing_page")),
        "type_raw": safe_join(it.get("type") or [], key="name") if isinstance(it.get("type"), list) else (
    it.get("type").get("name") if isinstance(it.get("type"), dict) else str(it.get("type")).strip() if it.get("type") else None
),

        "levels_raw": safe_join(it.get("levels") or []),
        "categories_raw": safe_join(it.get("categories") or []),
        "contents":  it.get("contents") or "",
        "_ingest_ts": datetime.utcnow().isoformat(timespec="seconds"),
        "_run_label": RUN_LABEL,
        "_source":    "the_muse_api",
    })

schema = T.StructType([
    T.StructField("job_id", T.LongType(), True),
    T.StructField("title_raw", T.StringType(), True),
    T.StructField("company", T.StringType(), True),
    T.StructField("location_raw", T.StringType(), True),
    T.StructField("post_date", T.StringType(), True),
    T.StructField("url", T.StringType(), True),
    T.StructField("type_raw", T.StringType(), True),
    T.StructField("levels_raw", T.StringType(), True),
    T.StructField("categories_raw", T.StringType(), True),
    T.StructField("contents", T.StringType(), True),
    T.StructField("_ingest_ts", T.StringType(), True),
    T.StructField("_run_label", T.StringType(), True),
    T.StructField("_source", T.StringType(), True),
])

df_new = spark.createDataFrame(rows, schema=schema)
df_new = df_new.filter(F.col("job_id").isNotNull())
display(df_new.limit(10))


/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071168-3207364523:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "_ingest_ts": datetime.utcnow().isoformat(timespec="seconds"),


job_id title_raw company location_raw post_date url type_raw levels_raw categories_raw contents _ingest_ts _run_label _source 17561167 Lead Cloud Automation Engineer (DevOps, AI/ML/Gen-AI) EPAM Systems Hyderabad, India 2025-01-09 https://www.themuse.com/jobs/epamsystems/lead-cloud-automation-engineer-devops-aimlgenai external Senior Level Data and Analytics EPAM is a leading global provider of digital platform engineering and development services. We are committed to having a positive impact on our customers, our employees, and our communities. We embrace a dynamic and inclusive culture. Here you will collaborate with multi-national teams, contribute to a myriad of innovative projects that deliver the most creative and cutting-edge solutions, and have an opportunity to continuously learn and grow. No matter where you are located, you will join a dedicated, creative, and diverse community that will help you discover your fullest potential. We are looking for a skilled Lead Cloud Automation Engineer to strengthen our Automation Engineering team. The role demands deep expertise in cloud infrastructure automation and DevOps, combined with solid knowledge in Infrastructure-as-Code (IaC) tools, generative AI, machine learning and AIOps. The successful candidate will be pivotal in enhancing our AIOps capabilities using generative AI models for anomaly detection, predictive maintenance and root cause analysis in a cloud environment. #LI-DNI #EasyApply Responsibilities Design, develop and maintain automated workflows for cloud infrastructure using Terraform and other IaC tools Enhance automation frameworks for infrastructure deployment across multiple cloud platforms Develop, manage and integrate GenAI service catalog code generation models with cloud automation pipelines Design and implement CI/CD pipelines to automate builds, testing and deployments Write and maintain automation scripts using Python, Bash or similar languages Design and develop generative AI models for AIOps applications using frameworks such as RAG with Langchain Utilize Vector Document Sources and Vector Database Sources in the context of generative AI Implement data streaming and manage cloud data lakes for AI model training Integrate generative AI models with existing AIOps platforms to enhance operational efficiency Collaborate with cross-functional teams to optimize cloud automation processes Research and implement new automation tools and techniques to improve efficiency Requirements Bachelor's or Master's degree in Computer Science, Engineering or a related field 8+ years of experience with IaC tools, including Terraform and CloudFormation Proven experience deploying generative AI models like RAG and understanding transformers Strong proficiency in Python and GenAI frameworks, such as RAG and Agentic Workflow Ability to build and manage Vector Document Sources using systems like Opensearch or Amazon Kendra Competency in creating Vector Database Sources with cloud or non-cloud databases Skills in data preparation, labeling and feature engineering for AI models Experience with cloud data lakes and vector databases for AI training Ability to create and manage the agentic workflow using ReAct Pattern or other GenAI platforms Expertise in developing CI/CD pipelines for a range of cloud automation cases Nice to have Familiarity with Cloud GenAI platforms like Bedrock Agents Experience integrating multi-agent systems for data aggregation in agentic workflows Knowledge of MLOps pipelines for deploying and monitoring AI models We offer Opportunity to work on technical challenges that may impact across geographies Vast opportunities for self-development: online university, knowledge sharing opportunities globally, learning opportunities through external certifications Opportunity to share your ideas on international platforms Sponsored Tech Talks & Hackathons Unlimited access to LinkedIn learning solutions Possibility to relocate to any EPAM office for short and long-t

In [0]:
full_table_name = f"{BRONZE_DB}.{BRONZE_TBL}"

if spark.sql(f"SHOW TABLES IN {BRONZE_DB} LIKE '{BRONZE_TBL}'").count() > 0:
    df_existing = spark.table(full_table_name)
    combined = df_existing.unionByName(df_new, allowMissingColumns=True)

    w = Window.partitionBy("job_id").orderBy(F.col("_ingest_ts").desc())
    bronze_final = (combined
                    .withColumn("_rn", F.row_number().over(w))
                    .filter(F.col("_rn")==1)
                    .drop("_rn"))
else:
    bronze_final = df_new

(bronze_final
 .write
 .mode("overwrite")             # safe because we de-duped above
 .format("delta")
 .saveAsTable(full_table_name))

log(f"Bronze table written: {full_table_name}")




/home/spark-dc2e7316-9107-44e2-b1ce-15/.ipykernel/2698/command-8803131332071165-604942667:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"[{datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC] {msg}")


[2025-08-27 23:04:05 UTC] Bronze table written: bronze_db.jobs_bronze


In [0]:
spark.sql(f"SELECT COUNT(*) AS rows FROM {full_table_name}").show()
spark.sql(f"""
SELECT post_date, COUNT(*) AS postings
FROM {full_table_name}
GROUP BY post_date
ORDER BY post_date DESC
LIMIT 10
""").show()


+----+
|rows|
+----+
|1212|
+----+

+----------+--------+
| post_date|postings|
+----------+--------+
|2025-08-27|      70|
|2025-08-26|      23|
|2025-08-25|      12|
|2025-08-24|       8|
|2025-08-23|       6|
|2025-08-22|       5|
|2025-08-21|      75|
|2025-08-20|      74|
|2025-08-19|      16|
|2025-08-18|       4|
+----------+--------+



In [0]:
display(spark.sql("SELECT * FROM bronze_db.jobs_bronze LIMIT 10"))


job_id title_raw company location_raw post_date url type_raw levels_raw categories_raw contents _ingest_ts _run_label _source 3104241 Merchandise and Stocking Associate Walmart Albuquerque, NM 2025-08-13 https://www.themuse.com/jobs/walmart/merchandise-and-stocking-associate-68d372 external Mid Level Data and Analytics Position Summary... Do you like to work on your feet and keep things neat and organized? Our merchandising & stocking associates connect all of the dots to make sure members can find everything they have on their shopping list. From every day needs to special occasions, members need you, as their mini tour guide, to take them that special product. Depending on the shift you work, your job could include moving inventory in the backroom, unloading trucks, fulfilling club pick-up orders or helping members while stocking shelves. From unloading trucks in the summer to filling ice cream in the freezer, this fast-paced job can be physically demanding - it's like being paid to go to the gym! Be a part of a great team with a common goal - making sure members can find more of what they love, for less. You will sweep us off our feet if: • You thrive in fast-paced environments • You keep member satisfaction as your top priority • You're comfortable with change and quickly adapt to different work scenarios • You're a problem solver who tackles obstacles head-on to ensure each task is completed with excellence • You are able to pick up boxes and other heavy objects weighing more than 25 pounds You will make an impact by: • Promptly unloading trucks • Assisting fellow associates as needed throughout the store • Sorting and stocking products on shelves and in the backroom • Engaging with vendors and drivers with a positive attitude • Maintaining a clean, neat, and member-ready area The merchandising & stocking associate role is a great way to start a fulfilling career at Sam's Club. Apply now! The above information has been designed to indicate the general nature and level of work performed in the role. It is not designed to contain or be interpreted as a comprehensive inventory of all duties, responsibilities and qualifications required of employees assigned to this job. The full job description can be made available as part of the hiring process. What you'll do... Provides member service by acknowledging the member and identifying member needs; providing guidance and support to members regarding self- service technology; assisting members with purchasing decisions; locating merchandise; resolving member issues and concerns; and promoting the company's products and services. Maintains safety of facility according to company policies and procedures by conducting safety sweeps; following forklift spotting procedures; following procedures for handling and disposing of hazardous materials; following company steel standard guidelines; and correcting and reporting unsafe situations to management. Maintains the sales floor and merchandise presentation in accordance with company policies and procedures by properly zoning the area; stocking, arranging and organizing merchandise; setting up, cleaning, and organizing product displays; removing damaged goods; signing and pricing merchandise according to company policies and procedures; identifying shrink and damages; and securing fragile and high-shrink merchandise. Monitors food and merchandise quality by ensuring product rotation, code dating, product recalls, and sanitation standards are followed according to company policies and procedures; and ensuring prominent display of promotional and seasonal merchandise. Complies with company policies, procedures, and standards of ethics and integrity by implementing related action plans; using the Open Door Policy; and applying these in executing business processes and practices. Completes work assignments and priorities by using policies, data, and resources; collaborating with managers, co-workers, customers, and other business partners; id